In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm
import os

import time
import numpy as np
import pandas as pd

In [2]:
#!cp restaurants.h5 restaurants_copy.h5
restaurants = pd.read_csv('restaurants.csv').set_index('Unnamed: 0')

In [4]:
def get_element(driver, xpath):
    for i in range(5):
        try:
            result = driver.find_element_by_xpath(xpath)
            return result
        except StaleElementReferenceException:
            time.sleep(.5)
    raise StaleElementReferenceException
    
for restaurant_index in tqdm(restaurants.index, total=len(restaurants)):
    
    result_filename = 'not_reviews/%s.h5' % restaurant_index
    if os.path.exists(result_filename):
        continue
    
    profile = webdriver.FirefoxProfile()
    profile.set_preference('permissions.default.image', 2)
    profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    
    driver = webdriver.Firefox(firefox_profile=profile)
    driver.get(restaurants.loc[restaurant_index]['restaurant_link'].replace('www.yelp.com/biz/', 'www.yelp.com/not_recommended_reviews/'))
    
    result = []
    while True:
        n = len(driver.find_elements_by_xpath('//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li'))

        for i in range(1, n+1):
            base_xpath = '(//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li)[%d]' % i

            try:
                src = get_element(driver, base_xpath + '//img[@class="photo-box-img"]').get_attribute('src')
                has_photo = 'default_avatars' not in src
            except:
                has_photo = False
                
            try:
                user_name = get_element(driver, base_xpath + '//li[@class="user-name"]').text
            except:
                user_name = ''
            try:
                localtion = get_element(driver, base_xpath + '//li[@class="user-location responsive-hidden-small"]').text
            except:
                localtion = ''
            try:
                friends = get_element(driver, base_xpath + '//li[@class="friend-count responsive-small-display-inline-block"]').text
                friends = int(friends.split(' ')[0])
            except:
                friends = 0
            try:
                reviews = get_element(driver, base_xpath + '//li[@class="review-count responsive-small-display-inline-block"]').text
                reviews = int(reviews.split(' ')[0])
            except:
                reviews = -1
            try:
                photos = get_element(driver, base_xpath + '//li[@class="photo-count responsive-small-display-inline-block"]').text
                photos = int(photos.split(' ')[0])
            except:
                photos = 0

            try:
                rating = get_element(driver, base_xpath + '//div[contains(@class, "i-stars")]').get_attribute('class')
                rating = int(rating.split(' ')[1][-1])
            except:
                rating = -1
            try:
                date = get_element(driver, base_xpath + '//span[@class="rating-qualifier"]').text
            except:
                date = ''
            try:
                review = get_element(driver, base_xpath + '//span[@class="js-content-toggleable hidden"]').get_attribute('innerHTML')
            except:
                try:
                    review = get_element(driver, base_xpath + '//span[@class="js-content-toggleable"]').get_attribute('innerHTML')
                except:
                    review = get_element(driver, base_xpath + '//div[@class="review-content"]/p').text


            result.append(dict(
                restaurant_id=restaurant_index,
                has_photo=has_photo,
                user_name=user_name,
                localtion=localtion,
                friends=friends,
                reviews=reviews,
                photos=photos,
                rating=rating,
                date=date,
                review=review,
            ))

        try:
            driver.find_element_by_xpath('//div[@class="ysection not-recommended-reviews review-list-wide"]//span[@class="pagination-label responsive-hidden-small pagination-links_anchor"]').click()
        except:
            break

    result = pd.DataFrame(result)
    result.to_hdf(result_filename, key='data', mode='w')
    driver.close()
    


  0%|          | 0/663 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 663/663 [6:08:52<00:00, 33.38s/it]  
